<h1><center>27 Day F10.7 Forecast Verification (Jan 2020 - Jul 2023)</center></h1>
<br>
<center>John Mayers, Physical Scientist <br> NOAA Space Weather Prediction Center</center>

The Space Weather Prediction Center produces 27 Day geomagnetic and radio forecasts found in the ["Weekly Highlights and 27-day Forecast"](https://www.swpc.noaa.gov/products/weekly-highlights-and-27-day-forecast). These forecast are not archived in a database and can only be found in these PDF documents. In order to assess forecast accuracy and skill, the forecasts have to be extracted from these PDF files into a readable format. This fist part of this program will extract the forecasts from the PDF files (found on page 4) using tabula-py then convert them into text files. The text files will then be read into Pandas dataframes for additional analysis. Once cleaned, these forecasts can be compared against observed values in order to obtain accuracy and skill scores. Observed data are archived in a database and have been made available in an .xlsx spreadsheet.

In [1]:
import pandas as pd
import tabula
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import re
    
from tabula.io import read_pdf
from tabulate import tabulate
from tabula.io import convert_into
from tqdm import tqdm
from datetime import datetime, timedelta
from numpy import nan
from matplotlib.pyplot import figure
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

<h3>Navigating to files</h3>

**Step 1**: Change the working directory

In [2]:
#change working directory to location of pdf files and set to variable
    
os.chdir('C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/')
directory = 'C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/'
print(f'The current working directory is {os. getcwd()}.')

The current working directory is C:\Users\john.mayers\Documents\27_Day\Data\2020_23_WeeklyPDF.


**Step 2**: Confirm the number of PDF files in the directory to be converted

In [3]:
pdf_ls = []
for file in glob.glob("*.pdf"):
    pdf_ls.append(file)
pdf_ls

num_pdfs = len(pdf_ls)

print(f'The are {num_pdfs} PDF files in this folder.')

The are 2 PDF files in this folder.


<h3>Preparing to use tabula-py to convert PDF to TXT</h3>

The tabula-py wrapper has a specific format *(convert_into(input_filename, output_filename, pages=all)* to read PDF files and convert to TXT. Since hundreds of PDF files will need to be processed, loops will be needed to iterate through input and output filenames.

**Step 1**: Create iterable output filenames

In [4]:
#creating a list of output_filenames from PDF filenames in dir but with .txt extension

filenames = os.listdir() #list all PDF files in folder and save to variable
filenames = [i.split('.', 1)[0] for i in filenames] #remove file extension from list of files and resave to variable

ls=[]

for i in range(len(filenames)):
    w = os.path.join(directory + filenames[i]+ ".txt") #concat directory, filenames with .txt extension
    ls.append(w)
    
#ls

['C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2314.txt',
 'C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2387.txt']

**Step 2**: Create iterable input filenames

In [5]:
#creating a list of input_filenames

ls1=[]

for i in range(len(filenames)):
    r = os.path.join(directory + filenames[i]+ ".pdf") #concat directory, filenames, with .pdf extension
    ls1.append(r)

#ls1

['C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2314.pdf',
 'C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2387.pdf']

<h3>Converting PDF to TXT using tabula-py</h3>

**Step 1**: Using tabula-py, iterate through the input and output files created in the previous step to convert PDF files to TXT files. Note each PDF file can take up to 60 seconds to be converted.

In [6]:
#using tabula-py convert_into() function to convert PDF files in dir to TXT

for pdf in tqdm(directory):
    for i in range(len(ls)):
        convert_into(ls1[i], ls[i], pages=4) #iterating through input and output filenames from lists above

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [01:26<00:00,  1.42s/it]


**Step 2**: Verify all PDF files were converted to TXT files

In [7]:
#verifying conversion was successful

num_txt = len(glob.glob1(directory,"*.txt"))

if num_pdfs == num_txt:
    print("All PDF files were successfully converted to TXT files")
else: 
    print("Some PDF files were not successfully converted")

All PDF files were successfully converted to TXT files


<h3>Converting all TXT files to Pandas Dataframe</h3>

In [8]:
f=[]

for file in os.listdir(directory):
    if file.endswith(".txt"):
        f.append(file)
print(f)

['prf2314.txt', 'prf2387.txt']


Looking at the first couple of TXT files that were converted from the first PDF file

In [9]:
file1 = open(f[0], 'r')
print(file1.read())
file1.close()

06 Jan,72,12,4,,20 Jan,70,5,2,
07,72,5,2,,21,70,5,2,
08,72,8,3,,22,70,5,2,
09,72,8,3,,23,70,5,2,
10,72,8,3,,24,70,5,2,
11,72,5,2,,25,71,5,2,
12,71,5,2,,26,72,5,2,
13,70,5,2,,27,72,5,2,
14,70,12,4,,28,72,5,2,
15,70,12,4,,29,72,5,2,
16,70,5,2,,30,72,5,2,
17,70,5,2,,31,72,5,2,
18,70,5,2,,01 Feb,72,8,3,
19,70,5,2,,,,,,



In [10]:
file2 = open(f[1], 'r')
print(file2.read())
file2.close()

24 May 0749,WATCH: Geomagnetic Storm Category G2 predicted,
24 May 1812,ALERT: Electron 2MeV Integral Flux >= 1000pfu,24/1805
25 May 1243,"CONTINUED ALERT:
Electron 2MeV Integral Flux >= 1000pfu",24/1805
26 May 1203,WARNING: Geomagnetic Sudden Impulse expected,26/1250 - 1330
26 May 1318,SUMMARY: Geomagnetic Sudden Impulse,26/1252
26 May 1621,WARNING: Geomagnetic K = 4,26/1625 - 2359
26 May 1622,"CONTINUED ALERT:
Electron 2MeV Integral Flux >= 1000pfu",24/1805
26 May 1821,EXTENDED WARNING: Geomagnetic K = 4,26/1625 - 27/0559
26 May 1822,WARNING: Geomagnetic K = 5,26/1825 - 2359
26 May 1830,ALERT: Geomagnetic K = 4,26/1828
26 May 2028,ALERT: Geomagnetic K = 5,26/2028
27 May 0244,EXTENDED WARNING: Geomagnetic K = 4,26/1625 - 27/1200
27 May 0245,WARNING: Geomagnetic K = 5,27/0244 - 0900
29 May 0024,ALERT: Type II Radio Emission,28/2303
29 May 0025,ALERT: Type IV Radio Emission,28/2301
29 May 0043,WARNING: Proton 10MeV Integral Flux > 10pfu,29/0042 - 1200
29 May 0319,ALERT: Proton Event 10M

In some instances the table of interest appears on page 5 (not 4). Continue to run the notebook to resolve this issue.

In [71]:
readfiles =[]
for txtfile in f:
    file = open(txtfile, 'r')
    readfiles.append(file.read())
    file.close()
# readfiles ## will preview all converted text files as a list

In [28]:
readfiles[0] # preview a select file by element

'06 Jan,72,12,4,,20 Jan,70,5,2,\n07,72,5,2,,21,70,5,2,\n08,72,8,3,,22,70,5,2,\n09,72,8,3,,23,70,5,2,\n10,72,8,3,,24,70,5,2,\n11,72,5,2,,25,71,5,2,\n12,71,5,2,,26,72,5,2,\n13,70,5,2,,27,72,5,2,\n14,70,12,4,,28,72,5,2,\n15,70,12,4,,29,72,5,2,\n16,70,5,2,,30,72,5,2,\n17,70,5,2,,31,72,5,2,\n18,70,5,2,,01 Feb,72,8,3,\n19,70,5,2,,,,,,\n'

In [29]:
len(readfiles[0]) # a correct table will have about 300 characters

317

In [30]:
len(readfiles[1]) # an incorrect table

1247

If length of file has more than 350 characters, it is likely the wrong table. We can use that as a filter.

In [31]:
filtered =[]

for s in range(len(readfiles)):
    if (len(readfiles[s]) > 350):
        filtered.append(s)
filtered = [int(i) for i in filtered] # these are the incorrect files corresponding to the element in the list. 
filtered

[1]

Here are the filenames that need to be re-processed with the correct page number using the convert_into function. More than likely this will be page 5.

In [42]:
fil = [] # locating the filename of the PDF with the problem
for val in filtered:
    fil.append(ls[val])
fil

['C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2387.txt']

Delete the incorrect text files

In [33]:
for txtfile in fil:
    os.remove(txtfile)

In [43]:
txt_ls = []
for file in glob.glob("*.txt"):
     txt_ls.append(file)

num_txt = len(txt_ls)

num_txt = len(glob.glob1(directory,"*.txt"))

if num_pdfs == num_txt:
    print("Try again")
else: 
    print("The bad text files were successfully deleted")

The bad text files were successfully deleted


Rerun the conversion for these PDF files but on page 5

In [44]:
#creating a new list of output_filenames from PDF filenames in dir but with .txt extension

fil

['C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2387.txt']

In [59]:
fil[0]

'C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2387.txt'

In [67]:
#creating a new list of input_filenames

input_new=[]

for file in range(len(fil)):
    split = os.path.splitext(fil[file])
    input_new.append(split[file] + ".pdf")
input_new


['C:/Users/john.mayers/Documents/27_Day/Data/2020_23_WeeklyPDF/prf2387.pdf']

In [68]:
for pdf in tqdm(directory):
    for i in range(len(fil)):
        convert_into(input_new[i], fil[i], pages=5) #iterating through input and output filenames from lists above

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:43<00:00,  1.42it/s]


In [80]:
readfiles =[]
for txtfile in f:
    file = open(txtfile, 'r')
    readfiles.append(file.read())
    file.close()

In [107]:
filtered =[]

for ele in range(len(fil)):
    if (len(readfiles[ele]) > 350):
        filtered.append(ele)

        filtered = [int(i) for i in filtered] # these are the incorrect files corresponding to the element in the list. 
    
    if len(filtered) == 0:
        print("All tables are correct.")
    else:
        print(f'The table from {filtered} is incorrect.')
    

All tables are correct.


In [108]:
main_df = pd.DataFrame(pd.read_csv(f[0])) # this cell will not run if there is an issue with the text files
  
for i in range(1,len(f)):
    data = pd.read_csv(f[i],header=None)
    df = pd.DataFrame(data)
    main_df = pd.concat([main_df,df], axis=1)
main_df.head()

,06 Jan,72,12,4,Unnamed: 4,20 Jan,70,5,2,Unnamed: 9,0,1,2,3,4,5,6,7,8,9
0,7.0,72.0,5.0,2.0,NaN,21,70.0,5.0,2.0,NaN,31 May,72,5,2,NaN,14 Jun,78.0,5.0,2.0,NaN
1,8.0,72.0,8.0,3.0,NaN,22,70.0,5.0,2.0,NaN,01 Jun,72,18,4,NaN,15,78.0,5.0,2.0,NaN
2,9.0,72.0,8.0,3.0,NaN,23,70.0,5.0,2.0,NaN,02,72,15,4,NaN,16,80.0,20.0,5.0,NaN
3,10.0,72.0,8.0,3.0,NaN,24,70.0,5.0,2.0,NaN,03,72,8,3,NaN,17,80.0,10.0,3.0,NaN
4,11.0,72.0,5.0,2.0,NaN,25,71.0,5.0,2.0,NaN,04,72,8,3,NaN,18,80.0,5.0,2.0,NaN


<h3>Cleaning the Data for Analysis</h3>

<h4>Understanding the table structure</h4>

In [ ]:
main_copy = main_df.copy() # make a copy of the df

In [ ]:
print(f'This table has {main_copy.shape[0]} rows and {main_copy.shape[1]} columns.')

<h4>Looking at a subset of the table corresponding to the first forecast</h4>

In [ ]:
pdf1 = main_copy.iloc[:,0:10] #looking at the cols corresponding to the first PDF. Each PDF has 10 cols.
pdf1.head()

<h4>Fixing col lables</h4>

In [ ]:
main_copy.head()

Every 5th col needs to be dropped

In [ ]:
main = main_copy.loc[:, (np.arange(len(main_copy.columns)) + 1) % 5 != 0]
main.head()

Creating a list of lists that repeats col labels based on the number of tables in the main df, then flattening that list of lists into a large list to pass into the df as the new col labels

In [ ]:
new_cols = [['Date', 'Radio', 'Ap', 'Kp', 'Date', 'Radio', 'Ap', 'Kp']] 

k=int(len(main.columns)/8)

res = [ele for ele in new_cols for i in range(k)] #list of lists of col headers

In [ ]:
flat_list = [item for sublist in res for item in sublist] # flattening into 1 large list

In [ ]:
len(main.columns)

In [ ]:
row = main.columns # to be transferred into first row
row

In [ ]:
main.loc[-1] = row # adding a row and setting it to row

In [ ]:
main.index = main.index + 1 #shifting index

In [ ]:
main = main.sort_index() # sorting by index

In [ ]:
main.columns = flat_list # setting new col labels

In [ ]:
main.head()

<h3>Inspecting individual forecasts from main df</h3>

In [ ]:
main.iloc[:,0:8]

In [ ]:
main.iloc[:,8:16]

In [ ]:
main.iloc[:,16:24]

Starting with col9, all values in row0 should be set to Nan. All these values should be replaced with NaN. They are erroneous.

In [ ]:
main.iloc[:1,8:] 

In [ ]:
main.iloc[:1,8:] = np.nan

In [ ]:
main.head()

<h3>Cleaning up dates</h3>

The dates present a problem, since only the first date is printed, followed by the day only, until the next col or if the month changes before then.

In [ ]:
main['Date']

In the first forecast, the start date is row0, col1

In [ ]:
c1 = main['Date'].iloc[0:1,:1]
c1

In the second forecast, and all following, the start date is row1, co12, col4, col6, etc.

In [ ]:
d1 = main['Date'].iloc[1:2,2:3]
d1

<h3>Building a new df with dates and F10 forecasts</h3>

In order to avoid some tricky programming to fill in the table, the approach will be to populate 27 days lists beginning with the start time of each forecast. 

<h4>Extracting start times</h4>

In [ ]:
c2 = pd.DataFrame(data=c1)
c2

In [ ]:
c2_ls = c2['Date'].tolist()
c2_ls

In [ ]:
c2_ls.append('2020')
c2_ls

In [ ]:
start = c2_ls[0] + ' ' + c2_ls[1]
start

In [ ]:
format ='%d %b %Y'
dt = datetime.strptime(start, format).date() #start date

In [ ]:
k = 27
 
res = []
 
for day in range(k):
    date = (dt + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)
res

<h4>Using Regex to find entries with months in dataframe</h4>

In [ ]:
import re

In [ ]:
from re import match

In [ ]:
ls = main['Date'].iloc[:,0:1].values.tolist()
ls

In [ ]:
regex = r"((\d+) [a-zA-Z]+)" # matches day and 3-digit month df
regex

In [ ]:
flat_list1 = [item for sublist in ls for item in sublist]

In [ ]:
new_list = [item for item in flat_list1 if not(pd.isnull(item)) == True] # removing nan in list

In [ ]:
new_list1 = [str(i) for i in new_list] # converting each element to a string
new_list1

In [ ]:
e =list(filter(lambda x: match(regex, x), new_list1))
e

In [ ]:
e.append('2020')
e

In [ ]:
start = e[0] + ' ' + e[1]
start

In [ ]:
format ='%d %b %Y'
dt = datetime.strptime(start, format).date() #start date

<h4>With initial start date, populating 1st forecast dates</h4>

In [ ]:
k = 27
 
res = []
 
for day in range(k):
    date = (dt + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)
res[:5]

So need to extract each forecast's start date

In [ ]:
k = 27
 
date1 = []
 
for day in range(k):
    date = (dt + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    date1.append(date)
date1[:5]

In [ ]:
begin = date1[0]
begin

In [ ]:
Begindate = datetime.strptime(begin, "%d %b %Y")
Begindate

<h4>Manually calculating forecast start times from pattern</h4>

Pattern: Each successive forecast increases by 7 days from the previous forecast or n times 7 from the first forecast where n is the number of forecasts from the first.

In [ ]:
Begindate2 = Begindate + timedelta(days=7)
Begindate2

In [ ]:
Begindate3 = Begindate + timedelta(days=14)
Begindate3

In [ ]:
Begindate4 = Begindate + timedelta(days=21)
Begindate4

In [ ]:
Begindate5 = Begindate + timedelta(days=28)
Begindate5

<h4> Automatically calculating start times for n forecasts</h4>

In [ ]:
#iterate the number of forecasts
Begindate = datetime.strptime(begin, "%d %b %Y")
mult = list(range(0,num_pdfs*7,7))
mult = mult[1:]

for j in mult: #multiples of 7 starting with 7
    x = Begindate + timedelta(days=j)
    print(x)

In [ ]:
#iterate the number of forecasts
Begindate = datetime.strptime(begin, "%d %b %Y")
mult = list(range(0,num_pdfs*7,7)) #multiples of 7 
mult = mult[1:] #multiples of 7 starting with 7

ls=[]

for j in mult:
    x = Begindate + timedelta(days=j)
    ls.append(x)

<h4>List of dates corresponding to 2nd forecast</h4>

In [ ]:
k = 27
 
res = []
 
for day in range(k):
    date = (ls[0] + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)

<h4>List of dates corresponding to 3rd forecast</h4>

In [ ]:
k = 27
 
res = []
 
for day in range(k):
    date = (ls[1] + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)

<h4> Manually compiling all forecast dates into a list</h4>

In [ ]:
k = 27
 
res = []

format ='%d %b %Y'
dt = datetime.strptime(start, format).date() 
 
for day in range(k):
    date = (dt + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)
    
res
 
for day in range(k):
    date = (ls[0] + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)
res


for day in range(k):
    date = (ls[1] + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)
res

for day in range(k):
    date = (ls[2] + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)
res

for day in range(k):
    date = (ls[3] + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res.append(date)

In [ ]:
len(res)

<h4>Automatically compile n forecast dates into a list</h4>

In [ ]:
k = 27 #iterates through all elements of ls to append dates in correct order

res2=[]

for i in range(len(ls)):
    for day in range(k):
        date = (ls[i] + timedelta(days = day))
        date = date.strftime('%d %b %Y')
        res2.append(date)

In [ ]:
print(f'There are {len(res2)} dates. Recall the first forecast is missing.')

<h4>Adding in first forecast to list</h4>

In [ ]:
k = 27 #now let's append the first forecast 

res11 = []
 
for day in range(k):
    date = (dt + timedelta(days = day))
    date = date.strftime('%d %b %Y')
    res11.append(date)

for i in range(len(ls)):
    for day in range(k):
        date = (ls[i] + timedelta(days = day))
        date = date.strftime('%d %b %Y')
        res11.append(date)
res11[:5]

In [ ]:
print(f'There are {len(res11)} dates corresponding to {num_pdfs*27} rows from {num_pdfs} forecasts.') #matches number of forecasts

<h4>Extracting F10 forecast values from the main df and creating a new df</h4>

In [ ]:
f10_wide = main['Radio']
f10_wide

In [ ]:
f10_long_nan = f10_wide.melt()
f10_long_nan

In [ ]:
f10_long = f10_long_nan.dropna(axis=0) # delete all rows with nan

In [ ]:
f10_long = f10_long.drop(['variable'], axis=1)
f10_long

In [ ]:
f10_long = f10_long.rename(columns={"value": "Kp"})

In [ ]:
f10_long = f10_long.reset_index()

In [ ]:
f10_long = f10_long.drop(['index'], axis=1)

In [ ]:
final_df = pd.DataFrame(res11, columns=['res']) #res11 (dates) needs to be transformed into a pandas df

In [ ]:
final_df = final_df.rename(columns={"res": "Date"})

In [ ]:
final_df['Radio'] = f10_long

<h4>The final table that observed F10 can be added to for analysis</h4>

In [ ]:
final_df.head(32)

Above you can see that the after Feb 1, the date does not continue with 2 Feb, but with 13 Feb, which is the starting date in the next forecast.

<h3>Importing Observed F10 Data</h3><br/>
Observed F10.7 from Government of Canada.

In [ ]:
f10obs = pd.read_excel('C:/Users/john.mayers/Documents/27_Day/Data/noon_flux2020.xlsx')
f10obs.head()

In [ ]:
f10obs = f10obs[['Date.1','Noon Flux']] 
f10obs.head()

<h3>Matching observed F10 with forecast F10</h3>

<h4>Creating a subset of obs that match up to the days corresponding to the n forecasts</h4>

In [ ]:
Begindate + timedelta(days=7)

In [ ]:
f10obs.iloc[5:27+5,0:2] # obs corresponding forecast 1

In [ ]:
f10obs.iloc[12:12+27,0:2] # obs corresponding to forecast 2

In [ ]:
f10obs.iloc[19:19+27,0:2] # obs corresponding to forecast 3

<h4> Writing a loop to populate a list of obs corresponding to forecast dates </h4>

The pattern emerges... 7 gets added each iteration to the start and end position of the row

In [ ]:
list(range(0,num_pdfs*7,7)) # recalling multiples of 7

In [ ]:
dates =[]

for i in range(0,num_pdfs*7,7):
    r = f10obs.iloc[5 + (i):5 +(i) +27,0:2]
    dates.append(r)    

In [ ]:
dates =[] 

for i in range(0,num_pdfs*7,7):
    r = f10obs.iloc[5 + (i):5 +(i) +27,1:2] # just the values without dates
    dates.append(r)

In [ ]:
print(f'This "pandas list" has {len(dates)} elements but we need {num_pdfs *27 }, so we will flatten the list.')

In [ ]:
dates[0].values.tolist()

Interating through each pandas list to convert to a list.

In [ ]:
ll=[]

for l in range(num_pdfs):
    a = dates[l].values.tolist()
    ll.append(a)

Flattening the list twice

In [ ]:
flat=[]

for sublist in ll:
    for element in sublist:
        flat.append(element)

In [ ]:
flat2=[]

for sublist in flat:
    for element in sublist:
        flat2.append(element)

In [ ]:
print(f'There are now {len(flat2)} observed values corresponding to {num_pdfs*27} forecasts in the correct order which can now be merged into 1 df.')

<h4>Final Merge</h4>

In [ ]:
complete_df = final_df.copy()

In [ ]:
complete_df = complete_df.rename(columns={"Kp": "Forecast Kp"})
complete_df.head()

In [ ]:
complete_df['Observed F10'] = flat2
complete_df.head()

In [ ]:
complete_df = complete_df.rename(columns={"Radio": "Forecast F10"})
complete_df.head()

In [ ]:
#Rounding Observed Kp to nearest integer

complete_df['Observed F10'] = complete_df['Observed F10'].round()
complete_df.head()

<h3>Final Table for Analysis</h3>

In [ ]:
complete_df.dropna(how="any", inplace=True)

In [ ]:
complete_df['Forecast F10']= complete_df['Forecast F10'].astype('int')
complete_df['Observed F10']= complete_df['Observed F10'].astype('int')
complete_df.head()

<h3>Forecast Performance</h3>

In [ ]:
complete_df["Forecast Error"] = complete_df["Forecast F10"] - complete_df["Observed F10"] 
complete_df.head()

# negative under forecast
# positive over forecast

In [ ]:
complete_df['Abs Error'] = complete_df['Forecast Error'].abs()
complete_df.head()

In [ ]:
last = final_df['Date'].iloc[-1]

print(f'For the {num_pdfs} forecasts, beginning on {begin} and ending on {last}, the average forecast error was {sum(complete_df["Abs Error"])/len(complete_df)} sfu.')

In [ ]:
x = sum(complete_df["Forecast Error"])/len(complete_df)
y=abs(x)

if sum(complete_df["Forecast Error"])/len(complete_df) < 0:
    print(f'On average, F10 was underforecast by {x}.')
    
else:
    print(f'On average, F10 was overforecast by {y}.')
          

In [ ]:
max_err = complete_df[['Abs Error']].max().tolist()
print(f' The max forecast error was {max_err} sfu.')

In [ ]:
count5 = complete_df['Abs Error'][complete_df['Abs Error'] > 5].count()
count10 = complete_df['Abs Error'][complete_df['Abs Error'] > 10].count()
count20 = complete_df['Abs Error'][complete_df['Abs Error'] > 20].count()

In [ ]:
print(f'The forecast was off by more the 5 sfu {count5} times, 10 sfu {count10} times and more than 20 sfu, {count20} times.')

In [ ]:
perfect = complete_df['Forecast Error'].value_counts()[0]
print(f'The forecast was still perfect {perfect} times or {perfect/len(complete_df) * 100} percent of the time.')

In [ ]:
complete_df.describe()

In [ ]:
freq = complete_df['Abs Error'].value_counts()

In [ ]:
plt.scatter(freq.index, freq.values)

plt.xlabel("Forecast Error", size=10)
plt.ylabel("Frequency", size=10)
plt.title("Frequency and Magnitude of Errors for F10", size=15)

plt.show()

<h4> Fitting a Regression </h4>

In [ ]:
lr = Ridge()

In [ ]:
X=complete_df['Date']
X_axis=np.arange(len(X))
lr.fit(X_axis.reshape(-1,1), complete_df['Abs Error'])

In [ ]:
figure(figsize=(70, 40), dpi=100)

plt.bar(X_axis, complete_df['Abs Error'])
plt.plot(X_axis, lr.coef_*X_axis+lr.intercept_, color='red', linewidth=20)

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)


plt.xlabel("Days", size=50)
plt.ylabel("F10 Error", size=50)
plt.title("Daily F10.7 Forecast Errors for [test_data]",fontsize=75)
plt.legend(['Trend'], fontsize=50)

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 10))

color = 'tab:blue'
ax1.set_xlabel('Days')
ax1.set_ylabel('Observed F10.7', color=color)
ax1.plot(X_axis, complete_df['Observed F10'], color=color, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('F10.7 Forecast Error', color=color)
ax2.bar(X_axis, complete_df['Abs Error'], color=color, alpha=0.25)
ax2.tick_params(axis='y', labelcolor=color)

plt.title("Comparison of Observed F10.7 with Forecast Errors for [test_data]",fontsize=20)

fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize =(6, 5))
plt.boxplot(complete_df['Forecast Error'])


plt.ylabel(" Error", size=10)
plt.title("F10.7 Forecast Error for [test_data]",fontsize=15)

plt.show

<h4>Mean Absolute Percentage Error</h4>
Definition: The absolute value of the difference between the forecasted value and the actual value taken as a mean.

In [ ]:
actual   = complete_df['Observed F10']
forecast = complete_df['Forecast F10']
  
APE = [] # percentage error between the forecast and observed value
  
for day in range(len(actual)):
    per_err = (actual[day] - forecast[day]) / actual[day]
    per_err = abs(per_err)
    APE.append(per_err)
  
MAPE = sum(APE)/len(APE)

print(f'''
MAPE   : { round(MAPE, 2) }
MAPE % : { round(MAPE*100, 2) } %
''')

<h4> Accuracy (POD) </h4>
Definition: How many times the forecast was correct.

In [ ]:
ACC = perfect/len(actual)
ACC

<h4> False Alarm Ratio (FAR) </h4>
Definition: How many times the forecast was wrong.

In [ ]:
FAR = (len(complete_df['Forecast F10'])-perfect) / len(complete_df['Forecast F10'])
FAR

*Note: POD and FAR are typically used for categorical data rather than discrete data such as integers. The definitions of both have been applied libearlly in order to compute these values.*

<h4> Heidke Skill Score </h4>
Compares the accuracy of the forecasts against the accuracy of some reference model (coefficient of determination), normalized by a perfect model score of 1 against the same coefficient of determination.

In order to create a "skill score" we need to create a reference model based on the observed data points and determine its accuracy (Aref). A linear regression model "trend line" through all observed values will serve as a first guess model for this demonstration. Future efforts should be made to fit a more accurate model in order to extract more meaningful value from a skill score.

In [ ]:
#lr.fit(X_axis.reshape(-1,1), complete_df['Observed F10'])

In [ ]:
model = LinearRegression().fit(X_axis.reshape(-1,1), complete_df['Observed F10'])

In [ ]:
r_sq = model.score(X_axis.reshape(-1,1), complete_df['Abs Error'])
r_sq

In [ ]:
Aperf = 1
Aref = r_sq

SS = (ACC - Aref)/(Aperf-Aref) # skill score formula
SS

In [ ]:
if SS == 1:
    print("The forecast is perfect")
elif SS > 0:
    print("The forecast is skillful and better than some reference.")
else:
    print("The forecast is less skillful than some reference.")

Now let's define a good forecast as an error no greater than 5.

In [ ]:
print(f' Recall there are {count5} forecasts with errors greater than 5.')

And we will define an accuracy below 70% as a bad forecast.

In [ ]:
ACC5 = (len(actual)-count5)/len(actual)

if ACC5 == 1:
    print(f'The forecasts are perfect with an overall accuracy of {ACC5*100}%.')
elif ACC5 > .7:
    print(f'The forecasts are good with an overall accuracy of {ACC5*100}%.')
else:
    print(f'The forecasts are bad with an overall accuracy of {ACC5*100}%.')

<h3>Conclusions</h3>

This notebook successfully demonstrates proof of concept in assessing the accuracy of 27 Day F10.7 Forecasts using a test dataset. 